## Akhilesh Pant (AU FTCA: MCA)

## Facial Emotion Detection

In [ ]:
import cv2
import threading
import numpy as np
import mediapipe as mp
from deepface import DeepFace

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

# Global variable to store detected sentiment
current_sentiment = "Analyzing..."
lock = threading.Lock()  # To handle concurrency


def analyze_emotion(face_frame):
    """Analyze the emotion of the detected face using DeepFace."""
    global current_sentiment
    try:
        if face_frame.size == 0:
            return

        # Convert BGR to RGB
        face_rgb = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)

        # Resize for DeepFace (DeepFace recommends 224x224 for best results)
        face_resized = cv2.resize(face_rgb, (224, 224))

        # Perform emotion analysis
        result = DeepFace.analyze(face_resized, actions=['emotion'], enforce_detection=False)

        # Extract sentiment
        with lock:
            if isinstance(result, list) and len(result) > 0:
                current_sentiment = result[0]['dominant_emotion']
            else:
                current_sentiment = result.get('dominant_emotion', "Unknown")
        print(f"Detected Sentiment: {current_sentiment}")

    except Exception as e:
        print("Error analyzing sentiment:", e)
        with lock:
            current_sentiment = "Error"


while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Convert frame to RGB (MediaPipe requires RGB input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    results = face_detector.process(frame_rgb)

    if results.detections:
        for detection in results.detections:
            # Get bounding box
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x, y, width, height = int(bboxC.xmin * w), int(bboxC.ymin * h), \
                                  int(bboxC.width * w), int(bboxC.height * h)

            # Ensure valid face region
            x, y, width, height = max(0, x), max(0, y), min(w-x, width), min(h-y, height)

            # Extract face ROI safely
            face_roi = frame[y:y+height, x:x+width].copy()  # Copy to avoid issues

            if face_roi.size > 0 and width > 10 and height > 10:  # Avoid tiny faces
                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

                # Run emotion analysis in a separate thread
                thread = threading.Thread(target=analyze_emotion, args=(face_roi,))
                thread.daemon = True  # Ensures thread closes on program exit
                thread.start()

    # Display sentiment on video feed
    cv2.putText(frame, f"Sentiment: {current_sentiment}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show live feed
    cv2.imshow("Face Sentiment Analysis", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting...")
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
